In [1]:
import time
import random
import requests
import pandas as pd
import urllib.request
from urllib.parse import quote

import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver  # 동적크롤링
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
# visitJeju크롤링 데이터 합치기
visitJeju크롤링_1005= pd.read_csv("Data/visitJeju 크롤링(~1005방선문).csv", index_col=False)
visitJeju크롤링_1006_1010 = pd.read_csv("Data/visitJeju 크롤링(1006~1010).csv", index_col=False)
visitJeju크롤링_1011 = pd.read_csv("Data/visitJeju 크롤링(1011~).csv", index_col=False)

visitJeju디테일뷰에사용할설명_1011 = pd.read_csv('Data/visitJeju 디테일뷰에 사용할 설명(1011~).csv', index_col=False)



FileNotFoundError: [Errno 2] No such file or directory: 'Data/visitJeju 크롤링(~1005방선문).csv'

In [ ]:
visitJeju크롤링_1005 = pd.DataFrame(visitJeju크롤링_1005)
visitJeju크롤링_1006_1010 = pd.DataFrame(visitJeju크롤링_1006_1010)
visitJeju크롤링_1011 = pd.DataFrame(visitJeju크롤링_1011)


In [ ]:
# 데이터 연결
merged_df = pd.concat([visitJeju크롤링_1005, visitJeju크롤링_1006_1010, visitJeju크롤링_1011], ignore_index=True)

In [ ]:
merged_df[1070:1100]

,title,stars,mainTag,subTag,text,BodyRightText
1070,스마일모텔,NaN,#모텔 #숙소 #주차장,#관광호텔 #공항,신제주중심가에 위치해있으며 제주시내의 먹거리 골목과 한라수목원 및 민오름의 사계절마...,"소개-공항 근처에 있는 편안한 모텔, 숙박 업종-모텔/여관, 이용 시간-Checki..."
1071,엠버호텔,NaN,#호텔 #숙소 #4성급호텔,#양식레스토랑 #공공와이파이존 #주차장 #무장애여행 #편의시설 #관광호텔 #안전여행...,"공항과 5분 거리의 신제주 시내에 있다. 호텔에서 제공하는 조식뷔페는 한식, 양식,...","소개-제주도심 한가운데에 위치한 ""빛나는 보석"" Amber Hotel, 이용 시간-..."
1072,제주메이플호텔,NaN,#휴식 #숙소 #호텔,#모텔 #주차장 #공공와이파이존 #공항,홍강과 차로 10분이내 거리에 위치하고 있는 신제주 중심가에 위치한 숙소이다.\n3...,"소개-제주시내 중심가에 위치한 교통이 편리한 호텔, 이용 시간-Checkin : ~..."
1073,테마하우스,NaN,#펜션 #숙소 #휴양펜션,#민박 #바비큐 #해수욕장 #공항 #온돌방,"또한 넓은 주차장 옆에는 바비큐장이 마련되어 있는데 근처에 있는 마트에서 참숯, 석...","소개-파도가 웃음으로 반겨주는 펜션, 이용 시간-Checkin : 14:00 ~ C..."
1074,롯데시티호텔 제주,별점(5점만점에 4점),#호텔 #제주시내 #숙소,#4성급호텔 #공공와이파이존 #교통 #양식레스토랑 #수영장 #주차장 #관광호텔 #무...,"제주의 대표적 절경인 주상절리와 폭포, 그리고 돌하르방을 형상화하여 설계된 롯데시티...","소개-간결하며 실용적인 공간, 자연스러움과 편안함을 품은 곳, 이용 시간-Check..."
1075,중문빌리지펜션,NaN,#펜션 #숙소 #바비큐,#해수욕장 #휴양펜션 #힐링 #중문관광단지 #가족호텔,"중문 빌리지는 원룸형 레저 하우스 형식의 펜션이며, 허니문 객실, 한식, 양식객실 ...","소개-원룸형 레저하우스, 이용 시간-Checkin : 15:00 ~ Checkout..."
1076,서귀포 늘바다 애견동반펜션,NaN,#펜션 #숙소 #휴양펜션,#반려동물보호소 #독채 #바비큐 #계곡 #단체여행객 #가족 #해변 #체험 #반려동물...,우아한 통나무집 한채가 나만의 별장이 되는 특별함과 고향집같은 편안함이 있는 숙소이다.,"소개-100이상 수령의 소나무로 지어진 통나무별장, 이용 시간-Checkin : ~..."
1077,유캔들잇스튜디오,NaN,#캔들만들기 #이색체험 #반려동물,#반려동물동반입장 #혼저옵서개 #반려동물공간_기타,"유캔들잇 스튜디오는 캔들 공방으로, 원데이클래스를 통해 다양한 캔들을 경험할 수 있...","소개-유캔들잇 스튜디오는 캔들 공방으로, 원데이클래스를 통해 다양한 캔들을 경험할 ..."
1078,제주게스트하우스 하쿠나마타타,NaN,#숙소 #게스트하우스 #민박,#조식포함 #자연경관 #힐링쉼터 #힐링,"1인실, 3인실, 4인실로 구성되어 있다.\n1인실은 이케아 슈퍼싱글 메트리스가 놓...","소개-제주시 애월 올레 15코스 종점, 16코스 시작점에 위치한 게스트 하우스, 이..."
1079,호텔 화인 제주,NaN,#호텔 #숙소 #게스트하우스,#4성급호텔 #양식레스토랑 #비즈니스센터 #관광호텔 #무장애관광,"서귀포항, 한라산 등을 바라볼 수 있는 발코니 전경이 일품인 화인호텔\n\n서귀포항...","소개-서귀포시 5성급 호텔로서 서비스와 시설면에서 최고급을 지향, 이용 시간-Che..."


In [ ]:
merged_df = merged_df.drop_duplicates(['title'], keep='first')

In [ ]:
merged_df.duplicated().sum()

0

In [ ]:
merged_df

,title,stars,mainTag,subTag,text,BodyRightText
0,미유,NaN,#한림 #옹포리 #소품샵,#핸드메이드소품 #키링 #우산 #지갑 #쇼핑,숲속 별장처럼 꾸며진 소품 가게 미유는 일몰의 풍경이 아름다운 것으로 유명한 한림항...,"소개-숲속 별장처럼 꾸며진 소품 가게, 이용 시간-평일 : 11:00 ~ 18:00..."
1,마야블루,NaN,#쇼핑 #라탄 #원데이클래스,#관광기념품 #상점/상가,마야블루 제주에는 작지만 매력적인 공간들이 꽤나 많다. 마야블루는 제주 시내의 주택...,소개-마야블루는 제주 시내의 주택가 사이에 작은 간판으로 자신의 존재를 알리고 있다...
2,몸냥공작소,NaN,#공방 #기념품 #쇼핑,#관광기념품 #상점/상가,"애월읍 중산간마을인 유수암리에 위치한 뜨개공방, 뜨개소품가게이다. 모든 작품들은 직...","소개-귀엽고 제주스러운 유니크한 소품가게, 상세 정보-토, 일, 공휴일 휴무, 이용..."
3,은인마켙,NaN,#함덕 #소품샵 #식기,#잡화,크지 않고 아담한 공간이지만 제법 다양한 소품들이 있고 둘러보기에 좋다. 사장님의 ...,"소개-자개, 유리 등 다양한 식기류를 만나볼 수 있는 소품샵, 상세 정보-매주 수요..."
4,뱅뱅와인마켓,NaN,#제주시내 #오라동 #와인,#샴페인,주기적인 와인 시음회를 열어 고객들에게 다양한 와인을 맛볼 기회를 제공하는 곳이다....,"소개-공항 인근에 위치한 대형 와인판매점, 이용 시간-평일 : 10:30 ~ 21:..."
...,...,...,...,...,...,...
2637,제주돌문화공원,별점(5점만점에 5점),#테마공원 #커플 #흐림,#겨울 #실내관광지 #어트랙션 #무장애관광 #공영관광지,제주돌문화공원은 돌의 고향인 제주도의 전신과 체계적인 돌 문화를 보여 주는 박물관이...,"소개-제주의 바람과 신화를 머금은 아름다운 돌을 감상하다, 상세 정보-휴일 : 매주..."
2638,검멀레동굴,별점(5점만점에 5점),#경관/포토 #친구 #커플,#부모 #아이 #맑음 #동굴 #자연경관 #포토스팟 #어린이,우도에서 가장 유명한 해변 중 하나인 검멀레 해변은 검은 모레가 있는 해변으로 알려...,소개-우도 8경 중 7경에 속하는 검은 모래가 있는 해안과 특이한 지형의 검멀레동굴...
2639,제주러브랜드,별점(5점만점에 4점),#커플 #테마공원 #가을,#밤 #실내관광지 #어트랙션 #무장애관광,제주러브랜드는 성을 주제로 한 테마조각공원으로 예술적으로 승화된 현대적 감각의 성 ...,"소개-성을 주제로 한 현대적 감각의 테마조각공원, 상세 정보-연중무휴/ 매표 마감 ..."
2640,한라산영실코스,별점(5점만점에 5점),#부모 #커플 #혼자,#친구 #경관/포토 #오름 #사계절 #자연경관 #체험 #포토스팟 #한라산 #언택트 ...,"백록담까지의 구간은 안전상의 문제로 통제되어 갈 수 없지만, 영실기암과 울창한 숲,...","소개-가장 짧지만 가장 아름다운, 한라산 등산코스, 상세 정보-기상 상황에 따라 입..."


In [ ]:
merged_df.to_csv("Data/visitJeju크롤링전체데이터.csv", index=False)

In [ ]:
visitJeju크롤링전체데이터.to_json('Data/제주도관광지데이터.json',orient='records', force_ascii=False)

In [7]:
visitJeju크롤링전체데이터= pd.read_csv('Data/visitJeju크롤링전체데이터.csv', index_col=False)

In [8]:
visitJeju크롤링전체데이터

,title,stars,mainTag,subTag,text,BodyRightText
0,미유,NaN,#한림 #옹포리 #소품샵,#핸드메이드소품 #키링 #우산 #지갑 #쇼핑,숲속 별장처럼 꾸며진 소품 가게 미유는 일몰의 풍경이 아름다운 것으로 유명한 한림항...,"소개-숲속 별장처럼 꾸며진 소품 가게, 이용 시간-평일 : 11:00 ~ 18:00..."
1,마야블루,NaN,#쇼핑 #라탄 #원데이클래스,#관광기념품 #상점/상가,마야블루 제주에는 작지만 매력적인 공간들이 꽤나 많다. 마야블루는 제주 시내의 주택...,소개-마야블루는 제주 시내의 주택가 사이에 작은 간판으로 자신의 존재를 알리고 있다...
2,몸냥공작소,NaN,#공방 #기념품 #쇼핑,#관광기념품 #상점/상가,"애월읍 중산간마을인 유수암리에 위치한 뜨개공방, 뜨개소품가게이다. 모든 작품들은 직...","소개-귀엽고 제주스러운 유니크한 소품가게, 상세 정보-토, 일, 공휴일 휴무, 이용..."
3,은인마켙,NaN,#함덕 #소품샵 #식기,#잡화,크지 않고 아담한 공간이지만 제법 다양한 소품들이 있고 둘러보기에 좋다. 사장님의 ...,"소개-자개, 유리 등 다양한 식기류를 만나볼 수 있는 소품샵, 상세 정보-매주 수요..."
4,뱅뱅와인마켓,NaN,#제주시내 #오라동 #와인,#샴페인,주기적인 와인 시음회를 열어 고객들에게 다양한 와인을 맛볼 기회를 제공하는 곳이다....,"소개-공항 인근에 위치한 대형 와인판매점, 이용 시간-평일 : 10:30 ~ 21:..."
...,...,...,...,...,...,...
2550,제주돌문화공원,별점(5점만점에 5점),#테마공원 #커플 #흐림,#겨울 #실내관광지 #어트랙션 #무장애관광 #공영관광지,제주돌문화공원은 돌의 고향인 제주도의 전신과 체계적인 돌 문화를 보여 주는 박물관이...,"소개-제주의 바람과 신화를 머금은 아름다운 돌을 감상하다, 상세 정보-휴일 : 매주..."
2551,검멀레동굴,별점(5점만점에 5점),#경관/포토 #친구 #커플,#부모 #아이 #맑음 #동굴 #자연경관 #포토스팟 #어린이,우도에서 가장 유명한 해변 중 하나인 검멀레 해변은 검은 모레가 있는 해변으로 알려...,소개-우도 8경 중 7경에 속하는 검은 모래가 있는 해안과 특이한 지형의 검멀레동굴...
2552,제주러브랜드,별점(5점만점에 4점),#커플 #테마공원 #가을,#밤 #실내관광지 #어트랙션 #무장애관광,제주러브랜드는 성을 주제로 한 테마조각공원으로 예술적으로 승화된 현대적 감각의 성 ...,"소개-성을 주제로 한 현대적 감각의 테마조각공원, 상세 정보-연중무휴/ 매표 마감 ..."
2553,한라산영실코스,별점(5점만점에 5점),#부모 #커플 #혼자,#친구 #경관/포토 #오름 #사계절 #자연경관 #체험 #포토스팟 #한라산 #언택트 ...,"백록담까지의 구간은 안전상의 문제로 통제되어 갈 수 없지만, 영실기암과 울창한 숲,...","소개-가장 짧지만 가장 아름다운, 한라산 등산코스, 상세 정보-기상 상황에 따라 입..."


In [12]:
print(visitJeju크롤링전체데이터['BodyRightText'][0])
print(visitJeju크롤링전체데이터['BodyRightText'][1])

소개-숲속 별장처럼 꾸며진 소품 가게, 이용 시간-평일 : 11:00 ~ 18:00 , 주말 : 11:00 ~ 18:00
소개-마야블루는 제주 시내의 주택가 사이에 작은 간판으로 자신의 존재를 알리고 있다. 이곳은 소품 판매와 함께 공방을 운영하면서 드림캐처와 캔들 등 직접 만들어 볼 수 있는 원데이 클래스도 진행하고 있다, 상세 정보-일요일 휴무, 이용 시간-평일 : 10:00 ~ 19:00 , 주말 : 10:00 ~ 19:00, 취급품목 기타-드림캐처, 캔들, 악세사리, 라탄 소품, 편의시설-카드결제,현금결제


In [14]:
visitJeju크롤링전체데이터['BodyRightText'].fillna('데이터없음', inplace=True)

In [19]:
visitJeju크롤링전체데이터

,title,stars,mainTag,subTag,text,BodyRightText
0,미유,NaN,#한림 #옹포리 #소품샵,#핸드메이드소품 #키링 #우산 #지갑 #쇼핑,숲속 별장처럼 꾸며진 소품 가게 미유는 일몰의 풍경이 아름다운 것으로 유명한 한림항...,"소개-숲속 별장처럼 꾸며진 소품 가게, 이용 시간-평일 : 11:00 ~ 18:00..."
1,마야블루,NaN,#쇼핑 #라탄 #원데이클래스,#관광기념품 #상점/상가,마야블루 제주에는 작지만 매력적인 공간들이 꽤나 많다. 마야블루는 제주 시내의 주택...,소개-마야블루는 제주 시내의 주택가 사이에 작은 간판으로 자신의 존재를 알리고 있다...
2,몸냥공작소,NaN,#공방 #기념품 #쇼핑,#관광기념품 #상점/상가,"애월읍 중산간마을인 유수암리에 위치한 뜨개공방, 뜨개소품가게이다. 모든 작품들은 직...","소개-귀엽고 제주스러운 유니크한 소품가게, 상세 정보-토, 일, 공휴일 휴무, 이용..."
3,은인마켙,NaN,#함덕 #소품샵 #식기,#잡화,크지 않고 아담한 공간이지만 제법 다양한 소품들이 있고 둘러보기에 좋다. 사장님의 ...,"소개-자개, 유리 등 다양한 식기류를 만나볼 수 있는 소품샵, 상세 정보-매주 수요..."
4,뱅뱅와인마켓,NaN,#제주시내 #오라동 #와인,#샴페인,주기적인 와인 시음회를 열어 고객들에게 다양한 와인을 맛볼 기회를 제공하는 곳이다....,"소개-공항 인근에 위치한 대형 와인판매점, 이용 시간-평일 : 10:30 ~ 21:..."
...,...,...,...,...,...,...
2550,제주돌문화공원,별점(5점만점에 5점),#테마공원 #커플 #흐림,#겨울 #실내관광지 #어트랙션 #무장애관광 #공영관광지,제주돌문화공원은 돌의 고향인 제주도의 전신과 체계적인 돌 문화를 보여 주는 박물관이...,"소개-제주의 바람과 신화를 머금은 아름다운 돌을 감상하다, 상세 정보-휴일 : 매주..."
2551,검멀레동굴,별점(5점만점에 5점),#경관/포토 #친구 #커플,#부모 #아이 #맑음 #동굴 #자연경관 #포토스팟 #어린이,우도에서 가장 유명한 해변 중 하나인 검멀레 해변은 검은 모레가 있는 해변으로 알려...,소개-우도 8경 중 7경에 속하는 검은 모래가 있는 해안과 특이한 지형의 검멀레동굴...
2552,제주러브랜드,별점(5점만점에 4점),#커플 #테마공원 #가을,#밤 #실내관광지 #어트랙션 #무장애관광,제주러브랜드는 성을 주제로 한 테마조각공원으로 예술적으로 승화된 현대적 감각의 성 ...,"소개-성을 주제로 한 현대적 감각의 테마조각공원, 상세 정보-연중무휴/ 매표 마감 ..."
2553,한라산영실코스,별점(5점만점에 5점),#부모 #커플 #혼자,#친구 #경관/포토 #오름 #사계절 #자연경관 #체험 #포토스팟 #한라산 #언택트 ...,"백록담까지의 구간은 안전상의 문제로 통제되어 갈 수 없지만, 영실기암과 울창한 숲,...","소개-가장 짧지만 가장 아름다운, 한라산 등산코스, 상세 정보-기상 상황에 따라 입..."


# 이미지 합치기

In [5]:
# visitJeju이미지 크롤링 데이터 합치기
visitJeju이미지크롤링_1005= pd.read_csv("Data/크롤링 데이터/visitJeju 이미지크롤링(~1005방선문).csv", index_col=False)
visitJeju이미지크롤링_1010= pd.read_csv("Data/크롤링 데이터/visitJeju 이미지크롤링(1006~1010).csv", index_col=False)
visitJeju이미지크롤링_1011= pd.read_csv("Data/크롤링 데이터/visitJeju 이미지크롤링(1011~).csv", index_col=False)
# visitJeju크롤링_1006_1010 = pd.read_csv("Data/visitJeju 크롤링(1006~1010).csv", index_col=False)

# visitJeju크롤링_1011 = pd.read_csv("Data/visitJeju 크롤링(1011~).csv", index_col=False)

# visitJeju디테일뷰에사용할설명_1011 = pd.read_csv('Data/visitJeju 디테일뷰에 사용할 설명(1011~).csv', index_col=False)



In [13]:
# 데이터프레임 만들기
visitJeju이미지크롤링_1005 = pd.DataFrame(visitJeju이미지크롤링_1005)
visitJeju이미지크롤링_1010 = pd.DataFrame(visitJeju이미지크롤링_1010)
visitJeju이미지크롤링_1011 = pd.DataFrame(visitJeju이미지크롤링_1011)

In [24]:
4903 + 16 + 9099

14018

In [16]:
visitJeju이미지크롤링_1005

,title,image
0,미유0,https://api.cdn.visitjeju.net/photomng/imgpath...
1,미유1,https://api.cdn.visitjeju.net/photomng/imgpath...
2,미유2,https://api.cdn.visitjeju.net/photomng/imgpath...
3,미유3,https://api.cdn.visitjeju.net/photomng/imgpath...
4,미유4,https://api.cdn.visitjeju.net/photomng/imgpath...
...,...,...
4898,방선문0,https://api.cdn.visitjeju.net/photomng/imgpath...
4899,방선문1,https://api.cdn.visitjeju.net/photomng/imgpath...
4900,방선문2,https://api.cdn.visitjeju.net/photomng/imgpath...
4901,방선문3,https://api.cdn.visitjeju.net/photomng/imgpath...


In [17]:
visitJeju이미지크롤링_1010

,title,image
0,법화사지0,https://api.cdn.visitjeju.net/photomng/imgpath...
1,법화사지1,https://api.cdn.visitjeju.net/photomng/imgpath...
2,법화사지2,https://api.cdn.visitjeju.net/photomng/imgpath...
3,법화사지3,https://api.cdn.visitjeju.net/photomng/imgpath...
4,북오름0,https://api.cdn.visitjeju.net/photomng/imgpath...
5,북오름1,https://api.cdn.visitjeju.net/photomng/imgpath...
6,북오름2,https://api.cdn.visitjeju.net/photomng/imgpath...
7,북오름3,https://api.cdn.visitjeju.net/photomng/imgpath...
8,북오름4,https://api.cdn.visitjeju.net/photomng/imgpath...
9,비치미오름0,https://api.cdn.visitjeju.net/photomng/imgpath...


In [18]:
visitJeju이미지크롤링_1011

,title,image
0,삼각봉0,https://api.cdn.visitjeju.net/photomng/imgpath...
1,삼각봉1,https://api.cdn.visitjeju.net/photomng/imgpath...
2,삼각봉2,https://api.cdn.visitjeju.net/photomng/imgpath...
3,삼각봉3,https://api.cdn.visitjeju.net/photomng/imgpath...
4,삼각봉4,https://api.cdn.visitjeju.net/photomng/imgpath...
...,...,...
9094,한라산영실코스8,https://api.cdn.visitjeju.net/photomng/imgpath...
9095,새별프렌즈0,https://api.cdn.visitjeju.net/photomng/imgpath...
9096,새별프렌즈1,https://api.cdn.visitjeju.net/photomng/imgpath...
9097,새별프렌즈2,https://api.cdn.visitjeju.net/photomng/imgpath...


In [25]:
visitJeju이미지크롤링 = pd.concat([visitJeju이미지크롤링_1005, visitJeju이미지크롤링_1010, visitJeju이미지크롤링_1011], ignore_index=True)

In [26]:
visitJeju이미지크롤링

,title,image
0,미유0,https://api.cdn.visitjeju.net/photomng/imgpath...
1,미유1,https://api.cdn.visitjeju.net/photomng/imgpath...
2,미유2,https://api.cdn.visitjeju.net/photomng/imgpath...
3,미유3,https://api.cdn.visitjeju.net/photomng/imgpath...
4,미유4,https://api.cdn.visitjeju.net/photomng/imgpath...
...,...,...
14014,한라산영실코스8,https://api.cdn.visitjeju.net/photomng/imgpath...
14015,새별프렌즈0,https://api.cdn.visitjeju.net/photomng/imgpath...
14016,새별프렌즈1,https://api.cdn.visitjeju.net/photomng/imgpath...
14017,새별프렌즈2,https://api.cdn.visitjeju.net/photomng/imgpath...


In [27]:
visitJeju이미지크롤링 = pd.DataFrame(visitJeju이미지크롤링)

In [28]:
# Title공통 접두사 추출
visitJeju이미지크롤링['category'] = visitJeju이미지크롤링['title'].str.extract(r'(\D+)', expand=False)
print(visitJeju이미지크롤링)

          title                                              image category
0           미유0  https://api.cdn.visitjeju.net/photomng/imgpath...       미유
1           미유1  https://api.cdn.visitjeju.net/photomng/imgpath...       미유
2           미유2  https://api.cdn.visitjeju.net/photomng/imgpath...       미유
3           미유3  https://api.cdn.visitjeju.net/photomng/imgpath...       미유
4           미유4  https://api.cdn.visitjeju.net/photomng/imgpath...       미유
...         ...                                                ...      ...
14014  한라산영실코스8  https://api.cdn.visitjeju.net/photomng/imgpath...  한라산영실코스
14015    새별프렌즈0  https://api.cdn.visitjeju.net/photomng/imgpath...    새별프렌즈
14016    새별프렌즈1  https://api.cdn.visitjeju.net/photomng/imgpath...    새별프렌즈
14017    새별프렌즈2  https://api.cdn.visitjeju.net/photomng/imgpath...    새별프렌즈
14018    새별프렌즈3  https://api.cdn.visitjeju.net/photomng/imgpath...    새별프렌즈

[14019 rows x 3 columns]


In [34]:
visitJeju이미지크롤링 = visitJeju이미지크롤링[['category','title','image']]
visitJeju이미지크롤링.to_csv("Data/visitJeju크롤링이미지전체데이터.csv", index=False)

In [35]:
visitJeju크롤링이미지전체데이터 = pd.read_csv("Data/visitJeju크롤링이미지전체데이터.csv", index_col=False)

In [37]:
visitJeju크롤링이미지전체데이터.to_json("Data/제주도관광지이미지데이터.json", orient='records', force_ascii=False)